In [ ]:
from __future__ import print_function
import pprint
pp = pprint.PrettyPrinter(indent=4)
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Lambda, Permute, Dropout, add, dot, concatenate, multiply
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re
import tensorflow as tf
from nltk.tokenize import word_tokenize
from keras.callbacks import ModelCheckpoi`nt
import datetime
from itertools import chain
from process_data import load_entities, save_pickle, load_pickle, load_kv_pairs, lower_list, vectorize, vectorize_kv, find_ngrams

In [ ]:
is_babi = False
if is_babi:
    train_data = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_train.txt', is_babi)
    test_data = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_test.txt', is_babi)
else:
    # N = 49900
    N = 500000000
    mem_maxlen = 100 # 1つのエピソードに関連しているKVの数に対する制限
    train_data = load_pickle('mov_task1_qa_pipe_train.pickle')[:N]
    test_data = load_pickle('mov_task1_qa_pipe_test.pickle')[:N]
    kv_pairs = load_pickle('mov_kv_pairs.pickle')
    train_k = load_pickle('mov_train_k.pickle')
    train_v = load_pickle('mov_train_v.pickle')
    test_k = load_pickle('mov_test_k.pickle')
    test_v = load_pickle('mov_test_v.pickle')
#     train_k = np.array([list(chain(*x))[:mem_maxlen] for x in train_k])
#     train_v = np.array([list(chain(*x))[:mem_maxlen] for x in train_v])
#     test_k = np.array([list(chain(*x))[:mem_maxlen] for x in test_k])
#     test_v = np.array([list(chain(*x))[:mem_maxlen] for x in test_v])
    train_k = np.array(train_k)
    train_v = np.array(train_v)
    test_k = np.array(test_k)
    test_v = np.array(test_v)
    entities = load_pickle('mov_entities.pickle')
    entity_size = len(entities)

vocab = set(entities + ['directed_by', 'written_by', 'starred_actors', 'release_year', 'has_genre', 'has_tags', 'has_plot'] 
                     + ['!directed_by', '!written_by', '!starred_actors', '!release_year', '!has_genre', '!has_tags', '!has_plot'] )
for _, q, answer in train_data + test_data:
    vocab |= set(q + answer)
vocab = sorted(vocab)
save_pickle(vocab, 'mov_vocab.pickle')
# vocab = load_pickle('mov_vocab.pickle')
stopwords = load_pickle('mov_stopwords.pickle')

w2i = dict((c, i) for i, c in enumerate(vocab))
i2w = dict((i, c) for i, c in enumerate(vocab))
save_pickle(w2i, 'mov_w2i.pickle')
save_pickle(i2w, 'mov_i2w.pickle')
# w2i = load_pickle('mov_w2i.pickle')
# i2w = load_pickle('mov_i2w.pickle')

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) #+ 1
# story_maxlen = max(map(len, (x for x, _, _ in train_data + test_data)))
query_maxlen = max(map(len, (x for _, x, _ in train_data + test_data)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
# print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training data:', len(train_data))
print('Number of test data:', len(test_data))
print('-')
print('Here\'s what a "data" tuple looks like (input, query, answer):')
print(train_data[0])

In [ ]:
train_k_max_memsize = max(map(len, (d for d in train_k)))
train_v_max_memsize = max(map(len, (d for d in train_v)))
test_k_max_memsize = max(map(len, (d for d in test_k)))
test_v_max_memsize = max(map(len, (d for d in test_v)))
# print(train_k_max_memsize, train_v_max_memsize)
# for i, (k,v) in enumerate(zip(train_k, train_v)):
#     print(len(k), len(v))
#     if i > 100: break
#     continue
#     if len(k) == 12082:
#         print(i)
#         pp.pprint(k[:199])
#         break

In [ ]:
print(find_ngrams(entities, 'the movie a boy and his dog, when was it released'.split(' '), 100))
print(find_ngrams(entities, tokenize('the movie a boy and his dog, when was it released'), 100))
print(find_ngrams(entities, tokenize('w.s. van dyke was the director on which movies?'), 100))
print(find_ngrams(entities, word_tokenize('w.s. van dyke was the director on which movies?'), 100))

In [ ]:
# def find_ngrams(token_dict, text, n):
#     # base case
#     if n <= 1:
#         return text
#     # tokens committed to output
#     saved_tokens = []
#     # tokens remaining to be searched in sentence
#     search_tokens = text[:]
#     # tokens stored until next ngram found
#     next_search = []
#     while len(search_tokens) >= n:
#         ngram = ' '.join(search_tokens[:n])
#         if ngram in token_dict:
#             # first, search previous unmatched words for smaller ngrams
#             sub_n = min(len(next_search), n - 1)
#             saved_tokens.extend(find_ngrams(token_dict, next_search, sub_n))
#             next_search.clear()
#             # then add this ngram
#             saved_tokens.append(ngram)
#             # then pop this ngram from the remaining words to search
#             search_tokens = search_tokens[n:]
#         else:
#             next_search.append(search_tokens.pop(0))
#     remainder = next_search + search_tokens
#     sub_n = min(len(remainder), n - 1)
#     saved_tokens.extend(find_ngrams(token_dict, remainder, sub_n))
#     return saved_tokens
# t = 'what films are about w.s. van dyke'
# # t = ['what', 'films', 'are', 'about', 'w.s', '.', 'van', 'dyke'] 
# # t = ['what', 'films', 'are', 'about', 'steven', 'brill']
# find_ngrams(entities, t.split(' '), 1000)

# def tokenize(sent):
#     '''Return the tokens of a sentence including punctuation.
#     >>> tokenize('Bob dropped the apple. Where is the apple?')
#     ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
#     '''
#     return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]
# tokenize('what films are about w.s. van dyke')
# tokenize('when was the movie Sleep, My Love released?')
# ts = 'when was the movie sleep, my love released?'.split(' ')
# print(find_ngrams(entities, ts, 100))

In [ ]:
# kv_pairs[1000:1050]
# # kv_pairs = load_pickle('mov_kv_pairs.pickle')
# # print(kv_pairs[1])
print(train_k[0])
print(train_v[0])
print(vec_train_k[0])
# stopwords

In [ ]:
w2i['what']

In [ ]:
label_list = []
for _,_,a in (train_data+test_data):
    if a[0] not in label_list: label_list.append(a[0])
    
#     for aa in a:
#         if aa not in label_list: label_list.append(aa)
print(len(label_list), label_list[:10])

In [ ]:
w2i_label = dict((c, i) for i, c in enumerate(label_list))
i2w_label = dict((i, c) for i, c in enumerate(label_list))

In [ ]:
def vectorize(data, w2i, story_maxlen, query_maxlen, w2i_label):
    Q, A = [], []
    for story, question, answer in data:
        # Vectroize question
        q = [w2i[w] for w in question if w in w2i]
        q = q[:query_maxlen]
        q_pad_len = max(0, query_maxlen - len(q))
        q += [0] * q_pad_len

#         y = np.zeros(len(w2i_label))
#         y[w2i_label[answer[0]]] = 1
        y = np.zeros(len(w2i))
        y[w2i[answer[0]]] = 1
#         for a in answer:
#             y[w2i[a]] = 1

        Q.append(q)
        A.append(y)
    
    Q = np.array(Q, dtype=np.uint32)
    A = np.array(A, dtype='byte')

    return Q, A

queries_train, answers_train = vectorize(train_data,
                                           w2i,
                                           0,
                                           query_maxlen, w2i_label)
queries_test, answers_test = vectorize(test_data,
                                            w2i,
                                            0,
                                            query_maxlen, w2i_label)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
# print('inputs_train shape:', inputs_train.shape)
# print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_test shape:', answers_test.shape)

In [ ]:
chk_id = 100
print(train_data[chk_id][1])
print([w2i[w] for w in train_data[chk_id][1] if w in w2i])
# print([i2w[w] for w in train_data[0][1]])
print(queries_train[chk_id])
print([ i2w[i] for i in queries_train[chk_id]])
pp.pprint(train_k[chk_id])
pp.pprint(train_v[chk_id])
print(vec_train_k[chk_id]) -
print(answers_train[chk_id], i2w_label[np.argmax(answers_train[chk_id])])

In [ ]:
# max_memory_num = 100
# vec_train_k = vectorize_kv(train_k, mem_maxlen, w2i)
# vec_train_v = vectorize_kv(train_v, mem_maxlen, w2i)
# vec_test_k = vectorize_kv(test_k, mem_maxlen, w2i)
# vec_test_v = vectorize_kv(test_v, mem_maxlen, w2i)
# print('vec_k', vec_train_k.shape)
# print('vec_v', vec_train_v.shape)
def vectorize_kv(data, max_mem_len, max_mem_size, w2i):
    all_vec_list = []
    for i, kv_list in enumerate(data):
        if i % 5000 == 0: print(i, '/', len(data))
        vec_list = []
        for kv in kv_list:
            vec = [w2i[e] for e in kv if e in w2i]
            vec = vec[:max_mem_len]
            mem_pad_len = max(0, max_mem_len - len(vec))
            vec = vec + [0] * mem_pad_len
            vec_list.append(vec)
        vec_list = vec_list[:max_mem_size]
        mem_pad_size = max(0, max_mem_size - len(vec_list))
        for _ in range(mem_pad_size):
            vec_list.append([0] * max_mem_len)
        all_vec_list.append(vec_list)

    return np.array(all_vec_list, dtype=np.uint32)

max_mem_len = 4
max_mem_size = 15
vec_train_k = vectorize_kv(train_k, max_mem_len, max_mem_size, w2i)
vec_train_v = vectorize_kv(train_v, max_mem_len, max_mem_size, w2i)
vec_test_k = vectorize_kv(test_k, max_mem_len, max_mem_size, w2i)
vec_test_v = vectorize_kv(test_v, max_mem_len, max_mem_size, w2i)

In [ ]:
def MemNNKV(mem_len, mem_size, query_maxlen, vocab_size, embd_size, answer_size):
    print('mem_size:', mem_size)
    print('q_max', query_maxlen)
    print('embd_size', embd_size)
    print('vocab_size', vocab_size)
#     print('entity_size', entity_size)
    print('-----------')

    # placeholders
    key = Input((mem_size, mem_len,), name='Key_Input')
    val = Input((mem_size, mem_len,), name='Val_Input')
    question = Input((query_maxlen,), name='Question_Input')
    print('key:', key.shape)

    # encoders
    # memory encoders
    # output: (None, mem_size, embd_size)
    shared_embd_A = Embedding(input_dim=vocab_size, output_dim=embd_size)

    # embed the question into a sequence of vectors
    # output: (None, query_maxlen, embd_size)
    question_encoder = Sequential(name='Question_Encoder')
    question_encoder.add(shared_embd_A)
#     question_encoder.add(Dropout(0.3))

    # encode input sequence and questions (which are indices)
    # to sequences of dense vectors
    key_encoded = shared_embd_A(key) # (None, mem_size, mem_len, embd_size)
    print('key_encoded', key_encoded.shape)
    key_encoded = Lambda(lambda x: K.sum(x, axis=2)) (key_encoded) #(None, mem_size, embd_size)
    print('key_encoded', key_encoded.shape)
    val_encoded = shared_embd_A(val) # (None, mem_size, embd_size)
    val_encoded = Lambda(lambda x: K.sum(x, axis=2)) (val_encoded)
    
    question_encoded = question_encoder(question) # (None, query_max_len, embd_size)
#     question_encoded = Lambda(lambda x: K.sum(x, axis=1)) (val_encoded)
    question_encoded = Lambda(lambda x: K.sum(x, axis=1)) (question_encoded) #(None, embd_size)
    print('q_encoded', question_encoded.shape)
    q= question_encoded
    for h in range(1):
        print('---hop', h)
        ph = dot([q, key_encoded], axes=(1, 2))  # (None, mem_size)
        print('ph', ph.shape)
        ph = Activation('softmax')(ph)
        o = multiply([ph, Permute((2, 1))(val_encoded)]) # (None, embd_size, mem_size)
        print('o', o.shape)
        o = Lambda(lambda x: K.sum(x, axis=2))(o) # (None, embd_size)
        print('o', o.shape)
        R = Dense(embd_size, input_shape=(embd_size,), name='R_Dense_h' + str(h+1))
        q = R(add([q,  o])) # (None, embd_size)
        print('q', q.shape)

#     answer = Dense(answer_size, name='last_Dense')(q) #(None, answer_size)
    answer = Dense(vocab_size, name='last_Dense')(q) #(None, vocab_size)
    print('answer.shape', answer.shape)
    preds = Activation('softmax')(answer)
    
    # build the final model
    model = Model([key, val, question], preds)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    return model

print('vec_test_k.shape', vec_train_k.shape)
print('vec_test_v.shape', vec_train_v.shape)
print('queries_train.shape', queries_train.shape)
print('ans', answers_train.shape)

In [ ]:
embd_size = 200
memnn_kv = MemNNKV(max_mem_len, max_mem_size, query_maxlen, vocab_size, embd_size, len(label_list))
print(memnn_kv.summary())
now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
model_path = 'saved_models/' + now + '_kvnn-weights-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
memnn_kv.fit([vec_train_k, vec_train_v, queries_train], answers_train,
          batch_size=64,
          epochs=30,
          callbacks=callbacks_list,
          validation_data=([vec_test_k, vec_test_v, queries_test], answers_test))

In [ ]:
memnn_kv.save('model_memnn_kv.h5')

## Predict

In [ ]:
model = load_model('demo_model_memnn_kv.h5') # heavy to run
# score = model.evaluate([vec_test_k, vec_test_v, queries_test], answers_test, verbose=1)
# score = model.evaluate([vec_train_k, vec_train_v, queries_train], answers_train, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
# pred = model.predict([vec_train_k, vec_train_v, queries_train], verbose=1) 
pred = load_pickle('pred.pickle')

In [ ]:
for i, (p, a) in enumerate(zip(pred, answers_train[:len(pred)])):
    pw = i2w[np.argmax(p)]
    pa = i2w[np.argmax(a)]
    if pw == pa:
        print(i, ' '.join(train_data[i][1]))
        print(pw)


In [ ]:
print('q.shape', queries_test.shape)
ret_predict = model.predict([vec_test_k, vec_test_v, queries_test], verbose=1)

In [ ]:
print('q:',test_data[0][1])
print('predict:',i2w[np.argmax(ret_predict[0])])
print('label:',i2w[np.argmax(answers_test[0])])

## Interactive


In [ ]:
# stopwords = load_pickle('mov_stopwords.pickle')

# tokenize a question
q = 'who directed blade runner'
q_tokens = word_tokenize(q)
q_tokens = find_ngrams(vocab, q_tokens, 100000)
print('q_tokens:', q_tokens)

# vectorize a question
q_pad_len = max(0, query_maxlen - len(q_tokens))
vec_q = [w2i[w] for w in q_tokens] + [0] * q_pad_len
vec_q = np.array(vec_q)
vec_q = np.reshape(vec_q, (1, len(vec_q)))
print('vec_q:', vec_q)

# get related kv
k_list, v_list = [], []
for w in q_tokens:
    if w not in stopwords:
        for kv_ind, (k, v) in enumerate(kv_pairs):
            if w in (k+v):
                k_list += k
                v_list += v
    else:
        print(w, 'in stopwords')

def _vec_kv(data, w2i, mem_maxlen):
    vec = [w2i[e] for e in data if e in w2i]
    vec += [0] * max(0, mem_maxlen - len(vec))
    vec = vec[:mem_maxlen]
    vec = np.array(vec)
#     vec = np.expand_dims(vec, axis=0)
    print('---', vec.shape)
    vec = np.reshape(vec, (1, 100))
    
    return vec
# print(k_list)
# vectroize kv
vec_k, vec_v = None, None
vec_k = _vec_kv(k_list, w2i, mem_maxlen)
vec_v = _vec_kv(v_list, w2i, mem_maxlen)
# vec_k = [w2i[e] for e in k_list if e in w2i]
# vec_k += [0] * max(0, mem_maxlen - len(vec_k))
# vec_k = vec_k[:mem_maxlen]
# vec_k = np.array(vec_k)
# vec_v = [w2i[e] for e in v_list if e in w2i]
# vec_v += [0] * max(0, mem_maxlen - len(vec_v))
# vec_v = vec_v[:mem_maxlen]
# vec_v = np.array(vec_v)
print(vec_k.shape)

int_predict = model.predict([vec_k, vec_v, vec_q], batch_size=1, verbose=1)     
print('q:',q)
print('predict:',i2w[np.argmax(int_predict[0])])
# print('label:',i2w[np.argmax(answers_test[0])])

## misc

In [ ]:
vocab = set()
for story, q, answer in (train_data + test_data):#[:100]:
#     print(story + q + answer)
    vocab |= set(story + q + answer)
vocab = sorted(vocab)
print(len(list(set(vocab))))
print(vocab[:10])

In [ ]:
vocab[:100]

In [ ]:
def MemNN(story_maxlen, query_maxlen, vocab_size, embd_size):
    # placeholders
    input_sequence = Input((story_maxlen,))
    question = Input((query_maxlen,))

    # encoders
    # embed the input sequence into a sequence of vectors
    input_encoder_m = Sequential()
    input_encoder_m.add(Embedding(input_dim=vocab_size,
                                  output_dim=embd_size))
    input_encoder_m.add(Dropout(0.3))
    # output: (samples, story_maxlen, embedding_dim)

    # embed the input into a sequence of vectors of size query_maxlen
    input_encoder_c = Sequential()
    input_encoder_c.add(Embedding(input_dim=vocab_size,
                                  output_dim=query_maxlen))
    input_encoder_c.add(Dropout(0.3))
    # output: (samples, story_maxlen, query_maxlen)

    # embed the question into a sequence of vectors
    question_encoder = Sequential()
    question_encoder.add(Embedding(input_dim=vocab_size,
                                   output_dim=embd_size,
                                   input_length=query_maxlen))
    question_encoder.add(Dropout(0.3))
    # output: (samples, query_maxlen, embedding_dim)

    # encode input sequence and questions (which are indices)
    # to sequences of dense vectors
    input_encoded_m = input_encoder_m(input_sequence)
    input_encoded_c = input_encoder_c(input_sequence)
    question_encoded = question_encoder(question) # (None, query_max_len, embd_size)

    # compute a 'match' between the first input vector sequence
    # and the question vector sequence
    # shape: `(samples, story_maxlen, query_maxlen)`
    match = dot([input_encoded_m, question_encoded], axes=(2, 2))
    match = Activation('softmax')(match) # (None, max_storylen, query_maxlen)

    # add the match matrix with the second input vector sequence
    response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
    response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)
    print('---')
    print('match',match.shape)
    print('input_c', input_encoded_c.shape)
    print('response', response.shape)
    # concatenate the match matrix with the question vector sequence
#     answer = concatenate([response, question_encoded])
    answer = response
    print('---')
    print('resp.shape', response.shape)
    print('q_enc.shape', question_encoded.shape)
    print('answer.shape', answer.shape)

    # the original paper uses a matrix multiplication for this reduction step.
    # we choose to use a RNN instead.
    answer = LSTM(32)(answer)  # (samples, 32)

    # one regularization layer -- more would probably be needed.
#     answer = Dropout(0.3)(answer)
    answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
    # we output a probability distribution over the vocabulary
    answer = Activation('softmax')(answer)

    # build the final model
    model = Model([input_sequence, question], answer)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

embd_size = 64
model = MemNN(story_maxlen, query_maxlen, vocab_size, embd_size)
print(model.summary())

# train
model_path = 'kvnn-weights-{ecposh:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=3,
          callbacks=callbacks_list
          validation_data=([inputs_test, queries_test], answers_test))

In [ ]:
model = Sequential()
model.add(Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))

model.compile('rmsprop', 'mse')
print(model.shape)
output_array = model.predict(input_array)